In [1]:
import pandas as pd

In [2]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
union = pd.read_csv('../Data/union.csv', sep=";")
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Target

El target de nuestro proyecto va a ser el campo **nretweets** (número de Retweets).

He decidido usar este campo por su posible valor para conocer si el tuit va a tener la repercusión deseada o no.

He llegado a este conclusión después de ver cómo se comporta dependiendo de la hora y del día de la semana. Aunque no existen variaciones muy grandes, sí que he podido comprobar que ciertas horas y ciertos días hay más inteacción dentro de la red social.
Donde se puede ver que los usuarios dan más a me gusta y retuitean. Por lo que seguramente exista un patrón en los datos, dado un patrón de conducta humana. Por ejemplo, en las horas de "sueño" existe menos interacción.



# Limpieza de los Datos

## Eliminar No interesantes

In [6]:
print(tweets.columns.tolist())

['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']


In [123]:
datosInteresantes = union[['day','hour','retweet','nreplies','nlikes']]
target = union[['nretweets']]
print(datosInteresantes.count())
print(target.count())

day         40867
hour        40867
retweet     40867
nreplies    40867
nlikes      40867
dtype: int64
nretweets    40867
dtype: int64


### Nulos

In [43]:
datosInteresantes.isnull().sum()

day         0
hour        0
retweet     0
nreplies    0
nlikes      0
dtype: int64

### Separamos datos categóricos

In [124]:
datosInteresantes.dtypes

day         int64
hour        int64
retweet      bool
nreplies    int64
nlikes      int64
dtype: object

El campo retweet es categórico (Verdadero o Falso)

In [129]:
datosInteresantes['retweet']

0         True
1        False
2         True
3        False
4        False
         ...  
40862    False
40863    False
40864    False
40865    False
40866    False
Name: retweet, Length: 40867, dtype: bool

In [126]:
from sklearn.preprocessing import OneHotEncoder

In [152]:
df_c = datosInteresantes.copy(deep = True)
enc_ohe = OneHotEncoder()
enc_ohe.fit(df_c[["retweet"]])

OneHotEncoder()

In [153]:
data_encoder = enc_ohe.transform(df_c[['retweet']]).toarray()
print(data_encoder)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Lo unimos al dataset

In [154]:
df_encoder = pd.DataFrame(data_encoder)
df_encoder.head()
df_c = df_c.join(df_encoder,lsuffix='_left', rsuffix='_right')
df_c.drop(['retweet'], axis=1, inplace=True)


In [155]:
df_c

,day,hour,nreplies,nlikes,0,1
0,4,0,0,0,0.00,1.00
1,2,22,368,5266,1.00,0.00
2,2,18,0,0,0.00,1.00
3,2,18,476,5638,1.00,0.00
4,2,7,546,8810,1.00,0.00
...,...,...,...,...,...,...
40862,2,13,90,961,1.00,0.00
40863,2,13,12,224,1.00,0.00
40864,2,13,12,240,1.00,0.00
40865,2,13,113,1357,1.00,0.00


## Separar datos Entrenamiento

In [38]:
from sklearn.model_selection import train_test_split

In [214]:
X_train, X_test, y_train, y_test = train_test_split(df_c, target, test_size=0.3)

## Nulos

In [200]:
X_train.isnull().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isnull'

# Entrenar Modelo

In [61]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

In [95]:
np.random.seed(5)
tf.random.set_seed(5)

In [159]:
model=Sequential()
model.add(Dense(10, activation='relu',input_dim=6))
model.add(Dense(12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error')

In [157]:
X_train = np.asarray(X_train).astype(np.int64)

In [160]:
model.fit(X_train, y_train,epochs=50) 

Epoch 1/50
894/894 [==============================] - 1s 746us/step - loss: 29885262.0000
Epoch 2/50
894/894 [==============================] - 1s 729us/step - loss: 29885284.0000
Epoch 3/50
894/894 [==============================] - 1s 739us/step - loss: 29885286.0000
Epoch 4/50
894/894 [==============================] - 1s 708us/step - loss: 29885276.0000
Epoch 5/50
894/894 [==============================] - 1s 702us/step - loss: 29885268.0000
Epoch 6/50
894/894 [==============================] - 1s 706us/step - loss: 29885304.0000
Epoch 7/50
894/894 [==============================] - 1s 713us/step - loss: 29885252.0000
Epoch 8/50
894/894 [==============================] - 1s 711us/step - loss: 29885256.0000
Epoch 9/50
894/894 [==============================] - 1s 683us/step - loss: 29885272.0000
Epoch 10/50
894/894 [==============================] - 1s 689us/step - loss: 29885264.0000
Epoch 11/50
894/894 [==============================] - 1s 681us/step - loss: 29885280.0000
Epoch 12

In [209]:
# día,hora,nreplies,nlikes,0_retweet,1_retweet
print(model.predict([[3,21,1,2,1,0]]))

[[0.9324487]]
